This notebook completes the PanTaGruEl model as found at [https://github.com/laurentpagnier/PanTaGruEl.jl](https://github.com/laurentpagnier/PanTaGruEl.jl)

# Import model

In [1]:
import os
import json

In [2]:
with open('../pantagruel.json') as f:
    model = json.load(f)

# Compute susceptance for all lines

In [3]:
for line in model['branch'].values():
    line['br_b'] = line['br_x'] / (line['br_x']**2 + line['br_r']**2)

# Fix missing bus locations

In [4]:
bus_location_fix = {
    "7321": "FR",
    "7578": "FR",
    "7015": "FR",
    "7586": "FR",
    "7788": "FR",
    "1482": "IT",
    "7822": "FR",
    "7380": "FR",
    "7336": "FR",
    "7333": "FR",
    "7327": "FR",
    "7175": "PT",
    "6988": "DK",
    "7334": "FR",
    "6873": "DK",
    "7821": "FR",
    "7575": "FR",
    "7750": "FR",
    "7337": "FR",
    "7715": "FR",
    "7339": "FR",
    "7741": "FR",
    "6949": "FR",
    "6943": "FR",
    "7322": "FR",
    "7326": "FR"
}

In [5]:
for (id, country) in bus_location_fix.items():
    model["bus"][id]["country"] = country

# Add a load at every bus

Some buses do not have an associated load in PanTaGruEl.

In [6]:
all_buses = set(model['bus'].keys())
len(all_buses)

4097

In [7]:
buses_with_loads = {str(load['load_bus']) for load in model['load'].values()}
len(buses_with_loads)

3998

Map between bus and load:

In [8]:
load_by_bus = {str(load['load_bus']): load['pd'] for load in model['load'].values()}

In [9]:
load_by_bus

{'7815': 1.61109049031271,
 '5024': 0.3560778425764904,
 '4656': 1.0243666380711383,
 '4682': 0.51052439893367,
 '6007': 0.5224167219286343,
 '1992': 0.1775631764991962,
 '5277': 1.4329939838238062,
 '510': 2.7132327279352006,
 '5378': 3.9198519261887346,
 '7490': 2.205587022514553,
 '5367': 2.5056697215386476,
 '4655': 0.6206592074214654,
 '5348': 1.3400787810004433,
 '6712': 0.17850202526268089,
 '7885': 0.5314420531563848,
 '7684': 0.7091687815482721,
 '2926': 1.3513316757127998,
 '5148': 1.8594123998551004,
 '6582': 0.6192410850609339,
 '4558': 1.6216528574326003,
 '2744': 1.0585780068822774,
 '2675': 0.5218383002131518,
 '4638': 0.4839104890041973,
 '1787': 2.170220895613956,
 '4600': 1.8604052755726337,
 '5426': 0.76785211761682,
 '5525': 0.5768884930788787,
 '6350': 0.3377911655664167,
 '7807': 0.2621964151440807,
 '2631': 0.8237653072999253,
 '2798': 1.5276903296218982,
 '3971': 0.5652833946851059,
 '148': 1.8807575084732269,
 '5524': 1.1886925817296892,
 '231': 0.7790980500637

Re-assign loads with IDs matching the buses, using the given value for existing loads, and otherwise zero.

In [10]:
model['load'] = {id: {
                    'index'   : int(id),
                    'load_bus': int(id),
                    'status'  : 1,
                    'pd'      : load_by_bus[id] if id in load_by_bus else 0.0,
                    'qd'      : 0
                } for id in all_buses}

# Add ENTSO-E generator names

Add temporary unique generator name for all gens of the model, as well as the generators country:

In [11]:
for gen_id, gen in model['gen'].items():
    bus_id = str(gen['gen_bus'])
    gen['name'] = model['bus'][bus_id]['name'] + ' ' + gen_id
    gen['country'] = model['bus'][bus_id]['country']

In [12]:
all_gen_names = {gen['name'] for gen in model['gen'].values()}

Import list of generators from ENTSO-E at a given reference time (January 2016)

In [13]:
import pandas as pd

In [14]:
entsoe_data_dir = os.path.expanduser('~/data/entso-e/raw')

In [15]:
entsoe_data = pd.DataFrame(pd.read_csv('%s/2016_01_ActualGenerationOutputPerGenerationUnit_16.1.A.csv' % entsoe_data_dir, sep='\t') \
    .groupby(['PowerSystemResourceName', 'MapCode', 'ProductionType', 'InstalledGenCapacity'])['ActualGenerationOutput'].mean()) \
    .reset_index()
entsoe_data

,PowerSystemResourceName,MapCode,ProductionType,InstalledGenCapacity,ActualGenerationOutput
0,ABOÑO 1,ES,Fossil Hard coal,341.7,155.520027
1,ABOÑO 2,ES,Fossil Hard coal,535.8,411.376075
2,ABTH7,GB,Fossil Hard coal,535.0,501.421135
3,ABTH8,GB,Fossil Hard coal,535.0,483.604071
4,ABTH9,GB,Fossil Hard coal,535.0,395.362667
...,...,...,...,...,...
1679,Zydowo H3,PL,Hydro Pumped Storage,54.0,2.414234
1680,iPower Solutions ltd,NIE,Fossil Oil,74.0,0.034341
1681,Åbyverket Örebro,SE,Biomass,106.0,29.132615
1682,Öresundsverket CHP G1,SE,Fossil Gas,294.0,210.140970


Determine country from MapCode column:

In [16]:
entsoe_country_map = {mapcode: mapcode for mapcode in entsoe_data['MapCode']}
for mapcode in entsoe_country_map.keys():
    if mapcode[0:2] == 'DE':
        entsoe_country_map[mapcode] = 'DE'
entsoe_data.insert(2, 'Country', list(map(lambda mapcode: entsoe_country_map[mapcode], entsoe_data['MapCode'])))

Filter generators that are in one of the model's country (ENTSO-E data also containes UK, Ireland, Sweden...)

In [17]:
model_countries = {gen['country'] for gen in model['gen'].values()}
model_countries

{'AL',
 'AT',
 'BA',
 'BE',
 'BG',
 'CH',
 'CZ',
 'DE',
 'DK',
 'ES',
 'FR',
 'GR',
 'HR',
 'HU',
 'IT',
 'LU',
 'ME',
 'MK',
 'NL',
 'PL',
 'PT',
 'RO',
 'RS',
 'SI',
 'SK',
 'XX'}

In [18]:
entsoe_data = entsoe_data[entsoe_data['Country'].isin(model_countries)]

### Switzerland

Manually examine ENTSO-E generators list:

In [19]:
CH_name_map = {
    'Beznau 943': ['Beznau 1', 'Beznau 2'],
    'Mühleberg 917': ['KKM Produktion'],
    'Leibstadt 908': ['Leibstadt'],
    'Gösgen 921': ['Kernkraftwerk Gösgen'],
    'Biasca 938': ['Centrale di Biasca'],
    'Veytaux 913': ['Usine de Veytaux'],
    'Bâtiaz 912': ['Usine de la Bâtiaz'],
    'Chamoson 914': ['Usine de Bieudron', 'Usine de Nendaz'],
    'Fionnay GD 911': ['Usine de Fionnay'],
    'Verbano 922': ['Centrale di Verbano'],
    'Robiei 936': ['Centrale di Robiei'],
    'Bitsch 916': ['Kraftwerk Bitsch'],
    'Bavona 935': ['Centrale di Bavona'],
    'Cavergno 923': ['Centrale di Cavergno'],
    'Mapragg 939': ['Mapragg - Gigerwald G1'],
    'Grimsel 909': ['KWO Produktion'],
    'Innertkirchen 918': ['KWO Produktion'],
    'Riddes 173': ['Stufe FMM Produktion Total'],
    'Fionnay FMM 910': ['Stufe FMM Produktion Total'],
    'Limmern 958': ['Limmern - Muttsee G1'],
    'Tierfehd 926': ['Tierfehd - Limmern GPSW'],
    'Olivone 925': ['AET Leventina']
}

In [20]:
CH_name_values = {name for list_of_names in CH_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [21]:
assert set(CH_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [22]:
assert CH_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [23]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'CH' and gen['name'] not in CH_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
11,Bärenburg 930,hydro_pure_storage,220.0
0,Castasegna 928,hydro_pure_storage,100.0
3,Ferrera 929,hydro_mixed,180.0
9,Foretaille 766,hydro_ror,98.0
5,Göschenen 919,hydro_pure_storage,160.0
6,Handeck 937,hydro_mixed,113.0
2,Laufenburg 920,hydro_ror,106.0
12,Löbbia 933,hydro_pure_storage,95.0
1,Pradella 934,hydro_pure_storage,288.0
4,Rothenbrunnen 932,hydro_pure_storage,127.0


ENTSO-E generators not in the list:

In [24]:
entsoe_data[(entsoe_data['Country'] == 'CH') & (~entsoe_data['PowerSystemResourceName'].isin(CH_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
776,KW Rheinfelden CH,Hydro Run-of-river and poundage,100.0
1580,Usine de Vallorcine,Hydro Water Reservoir,230.0


### France

In [25]:
FR_name_map = {
    'Belleville 757': ['BELLEVILLE 1', 'BELLEVILLE 2'],
    'Boctois 715': ['NOGENT 1', 'NOGENT 2'],
    'Cattenom 732': ['CATTENOM 1', 'CATTENOM 2', 'CATTENOM 3', 'CATTENOM 4'],
    'Chinon B 751': ['CHINON 1', 'CHINON 2', 'CHINON 3', 'CHINON 4'],
    'Chooz 518': ['CHOOZ 1', 'CHOOZ 2'],
    'Civaux 602': ['CIVAUX 1', 'CIVAUX 2'],
    'Coulange 657': ['CRUAS 1', 'CRUAS 2', 'CRUAS 3', 'CRUAS 4'],
    'Creys 690': ['BUGEY 2', 'BUGEY 3', 'BUGEY 4', 'BUGEY 5'],
    'Dampierre 708': ['DAMPIERRE 1', 'DAMPIERRE 2', 'DAMPIERRE 3', 'DAMPIERRE 4'],
    'Fessenheim 318': ['FESSENHEIM 1', 'FESSENHEIM 2'],
    'Flamanville 982': ['FLAMANVILLE 1', 'FLAMANVILLE 2'],
    'Golfech 552': ['GOLFECH 1', 'GOLFECH 2'],
    'Gravelines 723': ['GRAVELINES 1', 'GRAVELINES 2', 'GRAVELINES 3', 'GRAVELINES 4', 'GRAVELINES 5', 'GRAVELINES 6'],
    'Le Blayais 583': ['BLAYAIS 1', 'BLAYAIS 2', 'BLAYAIS 3', 'BLAYAIS 4'],
    'Paluel 718': ['PALUEL 1', 'PALUEL 2', 'PALUEL 3', 'PALUEL 4'],
    'Penly Poste 717': ['PENLY 1', 'PENLY 2'],
    'P.Cordier 670': ['ST ALBAN 1', 'ST ALBAN 2'],
    'St. Laurent 554': ['ST LAURENT 1', 'ST LAURENT 2'],
    'Tricastin 658': ['TRICASTIN 1', 'TRICASTIN 2', 'TRICASTIN 3', 'TRICASTIN 4'],
    'Cordemais Poste 986': ['CORDEMAIS 2', 'CORDEMAIS 3', 'CORDEMAIS 4', 'CORDEMAIS 5'],
    'Porcheville 711': ['PORCHEVILLE 1', 'PORCHEVILLE 2', 'PORCHEVILLE 3', 'PORCHEVILLE 4'],
    'Aramon 556': ['ARAMON 1', 'ARAMON 2'],
    'Villarodin 695': ['VILLARODIN 1', 'VILLARODIN 2'],
    'Le Pouget 553': ['POUGET 4'],
    'Brommat 758': ['BROMMAT 7'],
    'Le Cheylas 685': ['CHEYLAS 1', 'CHEYLAS 2'],
    'S. Bissorte 765': ['SUPER BISSORTE 1', 'SUPER BISSORTE 2', 'SUPER BISSORTE 3', 'SUPER BISSORTE 4', 'SUPER BISSORTE 5'],
    'Montezic poste et centrale 759': ['MONTEZIC 1', 'MONTEZIC 2', 'MONTEZIC 3', 'MONTEZIC 4'],
    'Revin 726': ['REVIN 1', 'REVIN 2', 'REVIN 3', 'REVIN 4'],
    'Guersac 987': ['SPEM CCG'],
    'Quartes 725': ['Pont-sur-Sambre'],
    'Braek 555': ['DK6-TG1', 'DK6-TG2', 'DK6-TV1', 'DK6-TV2'],
    'Ratier 754': ['AMFARD14', 'AMFARD15'],
    'Darse 610': ['COMBIGOLFE CCG', 'CYCOFOS PL1', 'CYCOFOS PL2'],
    'Ponteau 955': ['MARTIGUES PONTEAU 5', 'MARTIGUES PONTEAU 6'],
    'Custines 749': ['Croix-de-Metz'],
    'St. Avold 731': ['EMILE HUCHET 6', 'EMILE HUCHET 7', 'EMILE HUCHET 8'],
    'Bezaumont 729': ['BLENOD 5'],
    'Le Havre 603': ['HAVRE 4'],
    'La Palun 672': ['PROVENCE 5'],
    'Arrighi 709': ['ARRIGHI 1', 'ARRIGHI 2'],
    'Brennilis 985': ['BRENNILIS 1', 'BRENNILIS 2', 'BRENNILIS 3'],
    'Loscoat 983': ['DIRINON 1', 'DIRINON 2'],
    'Le Chesnoy 714': ['MONTEREAU 5', 'MONTEREAU 6'],
    'Villevaude 716': ['VAIRES 1', 'VAIRES 2', 'VAIRES 3'],
    'Sisteron 677': ['SISTERON 1', 'SISTERON 2'],
    'Le Chastang 612': ['CHASTANG 2', 'CHASTANG 3']
}

In [26]:
FR_name_values = {name for list_of_names in FR_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [27]:
assert set(FR_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [28]:
assert FR_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [29]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'FR' and gen['name'] not in FR_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
73,Airvault 597,Biomass,2.1260
72,Amiens 720,Biomass,2.8300
122,Amiens 721,Biomass,1.9030
158,Arnage 601,Biomass,14.3750
153,Aston 557,hydro_ror,104.0000
...,...,...,...
59,Vignol 668,Hydro,6.2600
95,Vincey 727,Hydro,1.5009
56,Vogelgrun 737,hydro_ror,141.0000
134,Volvon 762,Hydro,2.7200


ENTSO-E generators not in the list:

In [30]:
entsoe_data[(entsoe_data['Country'] == 'FR') & (~entsoe_data['PowerSystemResourceName'].isin(FR_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
15,AIGLE 6,Hydro Water Reservoir,146.0
118,BORT 1,Hydro Water Reservoir,118.0
119,BORT 2,Hydro Water Reservoir,114.0
245,COMBE D'AVRIEUX 1,Hydro Water Reservoir,123.0
548,FR-GA-MORANT1,Fossil Gas,401.0
569,GENNEVILLIERS 1,Fossil Gas,203.0
585,GRAND MAISON 1,Hydro Pumped Storage,152.0
586,GRAND MAISON 10,Hydro Pumped Storage,157.0
587,GRAND MAISON 11,Hydro Pumped Storage,157.0
588,GRAND MAISON 12,Hydro Pumped Storage,157.0


### Spain

In [31]:
ES_name_map = {
    'Asco 79': ['ASCO 1', 'ASCO 2'],
    'C.N. Almaraz 160': ['ALMARAZ 1', 'ALMARAZ 2'],
    'Sagunto 87': ['COFRENTES'],
    'La Robla 1005': ['ROBLA 1', 'ROBLA 2'],
    'Soto de Ribera 1013': ['SOTO RIB 2', 'SOTO RIB 3'],
    'P.G. Rodriguez 998': ['P.G.RODR 1', 'P.G.RODR 2', 'P.G.RODR 3', 'P.G.RODR 4'],
    'Teruel 77': ['TERUEL 1', 'TERUEL 2', 'TERUEL 3'],
    'Lada 1009': ['LADA IV'],
    'Compostilla I 1004': ['ANLLARES 1', 'COMPOSTI 4', 'COMPOSTI 5'],
    'Belesar 997': ['BELESAR 1', 'BELESAR 2', 'BELESAR 3'],
    'G. y Galan 152': ['GGALAN GEN'],
}

In [32]:
ES_name_values = {name for list_of_names in ES_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [33]:
assert set(ES_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [34]:
assert ES_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [35]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'ES' and gen['name'] not in ES_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
56,Aguayo 1010,Hydro,360.60
41,Aldeadávila II 166,hydro_ror,820.00
60,Alvarado 158,other_nl,150.00
66,Andújar 143,Waste,15.00
71,Aravaca 161,Hydro,61.95
...,...,...,...
48,Vandellos 80,Gas,39.58
82,Vilecha 1006,Hydro,34.68
17,Villalcampo 156,hydro_ror,881.00
9,Villamayor 154,other_nl,120.00


ENTSO-E generators not in the list:

In [36]:
entsoe_data[(entsoe_data['Country'] == 'ES') & (~entsoe_data['PowerSystemResourceName'].isin(ES_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
0,ABOÑO 1,Fossil Hard coal,341.7
1,ABOÑO 2,Fossil Hard coal,535.8
5,ACECA 3,Fossil Gas,386.0
6,ACECA4,Fossil Gas,372.6
16,ALDEA I G1,Hydro Water Reservoir,132.9
...,...,...,...
1601,VLLRINO 4G,Hydro Water Reservoir,138.7
1602,VLLRINO 5B,Hydro Pumped Storage,138.0
1603,VLLRINO 5G,Hydro Water Reservoir,148.9
1604,VLLRINO 6B,Hydro Pumped Storage,138.0


### Germany

In [37]:
DE_name_map = {
    'Brokdorf 210': ['Brokdorf'],
    'Gundremmingen 256': ['Gundremmingen B'],
    'Gundremmingen 257': ['Gundremmingen C'],
    'Isar 265': ['Isar 2'],
    'Philippsburg 232': ['KKW Philippsburg 2'],
    'Grohnde 209': ['Grohnde'],
    'Emsland 249': ['Emsland A'],
    'GKN 2 950': ['KKW Neckarwestheim 2'],
    'Boxberg 314': ['KW Boxberg Block N', 'KW Boxberg Block P', 'KW Boxberg Block Q', 'KW Boxberg Block R'],
    'Jänschwalde 312': ['KW Jänschwalde Block A', 'KW Jänschwalde Block B', 'KW Jänschwalde Block C',
                        'KW Jänschwalde Block D', 'KW Jänschwalde Block E', 'KW Jänschwalde Block F'],
    'Lippendorf 214': ['KW Lippendorf Block R', 'KW Lippendorf Block S'],
    'Neurath 238': ['Neurath A', 'Neurath B', 'Neurath C', 'Neurath D', 'Neurath E', 'Neurath F', 'Neurath G'],
    'Niederaußem 237': ['Niederaußem C', 'Niederaußem D', 'Niederaußem E', 'Niederaußem F',
                        'Niederaußem G', 'Niederaußem H', 'Niederaußem K (BoA 1)',
                        'Frimmersdorf P', 'Frimmersdorf Q',
                        'Weisweiler E', 'Weisweiler F', 'Weisweiler G', 'Weisweiler H'],
    'Schkopau 215': ['Schkopau A', 'Schkopau B'],
    'Buschhaus 213': ['Buschhaus'],
    'Schwarze Pumpe 313': ['KW Schwarze Pumpe Block A', 'KW Schwarze Pumpe Block B'],
    'Heilbronn 254': ['HKW Heilbronn Block 7'],
    'Rostock 304': ['Kraftwerk Rostock'], # incomplete,
    'Moorburg 319': ['HKW Moorburg Block A', 'HKW Moorburg Block B'],
    'Wilhelmshaven 204': ['Wilhelmshaven'],
    'Großkrotzenburg 272': ['Staudinger 5'], # incomplete
    'Bexbach 225': ['BEXBACH_A_GESAMT'],
    'Lippborg 248': ['BERGKAMEN_A'],
    'Ensdorf 948': ['Ensdorf 1', 'Ensdorf 3'],
    'Gersteinwerk 240': ['Gersteinwerk K2'],
    'Altbach 307': ['HKW Altbach/Deizisau Block 1', 'HKW Altbach/Deizisau Block 2'],
    'Reuter West 216': ['HKW Reuter West Block D', 'HKW Reuter West Block E'],
    'Kummerfeld 290': ['HKW Wedel Block 1', 'HKW Wedel Block 2'],
    'Ibbenbüren 309': ['Ibbenbüren B'],
    'Voerde 203': ['Kraftwerk Voerde Block A', 'Kraftwerk Voerde Block B'],
    'Scholven 243': ['Scholven B', 'Scholven C'], # incomplete
    'Ingolstadt 263': ['Ingolstadt 3', 'Ingolstadt 4'],
    'Vierraden 302': ['IKS Schwedt SE1 Block 1', 'IKS Schwedt SE2 Block 2']
}

In [38]:
DE_name_values = {name for list_of_names in DE_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [39]:
assert set(DE_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [40]:
assert DE_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [41]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'DE' and gen['name'] not in DE_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
65,Altenkleusheim 245,hydro_pure_ps,140.0
27,Altentreptow/Süd 972,Gas,75.0
25,BASF 270,fossil_coal_gas,460.0
106,BASF 271,fossil_coal_hard,870.0
103,Bergrheinfeld 281,hydro_pure_ps,160.0
...,...,...,...
0,Wengerohr 228,Hydro,16.4
78,Wengerohr 229,Hydro,13.6
26,Windpark Iven 954,Gas,15.1
85,Wolmirstedt 292,Gas,52.0


ENTSO-E generators not in the list:

In [42]:
entsoe_data[(entsoe_data['Country'] == 'DE') & (~entsoe_data['PowerSystemResourceName'].isin(DE_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
169,Block 1,Fossil Hard coal,136.0
170,Block 2,Fossil Hard coal,136.0
171,Block 3,Fossil Hard coal,690.0
172,Block AGuD,Fossil Gas,100.0
173,Block B,Fossil Gas,100.0
...,...,...,...
1659,Weisweiler VGT - BI. G,Fossil Gas,200.0
1660,Weisweiler VGT - BI. H,Fossil Gas,200.0
1662,Westfalen C,Fossil Hard coal,250.0
1663,Westfalen E,Fossil Hard coal,763.7


### Nuclear plants in other countries

In [43]:
jaslovske = [name for name in all_gen_names if name[-3:] == '377'][0]
name_map = CH_name_map | FR_name_map | ES_name_map | DE_name_map | {
    jaslovske: ['Bohunice TG31', 'Bohunice TG32', 'Bohunice TG41', 'Bohunice TG42'],
    'Borssele 846': ['Borssele 30'],
    'Tihange 521': ['TIHANGE 1N', 'TIHANGE 1S', 'TIHANGE 2', 'TIHANGE 3'],
    'Mochovce 378': ['Mochovce TG11', 'Mochovce TG12', 'Mochovce TG21', 'Mochovce TG22'],
    'Cernavoda 404': ['CNE_U1_RET_CA', 'CNE_U2_RET_CA'],
    'Kallo 515': ['DOEL 1', 'DOEL 2', 'DOEL 3', 'DOEL 4'],
    'Dukovany 176': ['EDUK_B1____', 'EDUK_B2____', 'EDUK_B3____', 'EDUK_B4____'],
    'Temelín 179': ['ETEM_G1____', 'ETEM_G2____'],
    'Kozloduy 418': ['NPP KZL G9', 'NPP KZL G10'],
    'Paks 782': ['PA_gép1', 'PA_gép2', 'PA_gép3', 'PA_gép4', 'PA_gép5', 'PA_gép6', 'PA_gép7', 'PA_gép8'],
}

In [44]:
name_values = {name for list_of_names in name_map.values() for name in list_of_names}

Total number of model generators, total number of ENTSO-E generators:

In [45]:
len(name_map), len(name_values)

(123, 275)

Check that the keys are part of the model's names:

In [46]:
assert set(name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [47]:
assert name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's nuclear generators not in the list:

In [48]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if 'nuclear' in gen['type'] and gen['name'] not in name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
1,Grafenrheinfeld 282,nuclear_cons,1275.0
0,Krško 886,nuclear,696.0


ENTSO-E nuclear generators not in the list:

In [49]:
entsoe_data[(entsoe_data['ProductionType'] == 'Nuclear') & (~entsoe_data['PowerSystemResourceName'].isin(name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
1186,S.M.GAROÑA,Nuclear,455.2
1353,TRILLO,Nuclear,1003.4
1588,VANDELLOS,Nuclear,1045.3


### Update the model

Add 'entsoe_names' field to the generators in the model and delete the temporary name:

In [50]:
for gen in model['gen'].values():
    gen['entsoe_names'] = name_map[gen['name']] if gen['name'] in name_map.keys() else []
    del gen['name']

# Add aggregated types

In [51]:
all_gen_types = {gen['type'] for gen in model['gen'].values()}

Manually define aggregated types:

In [52]:
types_aggregation_map = {
    "Hydro"                    : "hydro",
    "hydro_pure_storage"       : "hydro_storage",
    "hydro_pure_storage_cons"  : "hydro_storage",
    "hydro_pure_ps"            : "hydro_storage",
    "hydro_mixed"              : "hydro_storage",
    "hydro_mixed_cons"         : "hydro_storage",
    "hydro_ror"                : "hydro_ror",
    "fossil_brown_lignite"     : "coal",
    "fossil_brown_lignite_cons": "coal",
    "fossil_coal_hard"         : "coal",
    "fossil_coal_gas"          : "gas",
    "fossil_oil"               : "gas",
    "fossil_mixed"             : "gas",
    "fossil_coal_gas_cons"     : "gas",
    "Coal"                     : "coal",
    "Gas"                      : "gas",
    "Oil"                      : "gas",
    "nuclear"                  : "nuclear",
    "Nuclear"                  : "nuclear",
    "nuclear_cons"             : "nuclear",
    "other_nrenew"             : "other",
    "other_nrenew_cons"        : "other",
    "other_nl"                 : "other",
    "Geothermal"               : "other",
    "biomass"                  : "other",
    "Biomass"                  : "other",
    "Waste"                    : "gas",
    "waste_nr"                 : "gas"
}

In [53]:
aggregated_types = set(types_aggregation_map.values())
aggregated_types

{'coal', 'gas', 'hydro', 'hydro_ror', 'hydro_storage', 'nuclear', 'other'}

Check that all types are mapped: 

In [54]:
assert all_gen_types == set(types_aggregation_map.keys())

In [55]:
for gen in model['gen'].values():
    gen['aggregated_type'] = types_aggregation_map[gen['type']]

# Define expected usage

Choose 2016 as the reference year, as it matches with the model:

In [56]:
year = 2016

### Capacity

In [57]:
capacity_df = pd.read_csv('%s/%d_01_InstalledGenerationCapacityAggregated_14.1.A.csv' % (entsoe_data_dir, year + 1),
                          sep='\t', usecols=['AreaTypeCode', 'MapCode', 'ProductionType', 'AggregatedInstalledCapacity'])
capacity_df = capacity_df[(capacity_df['AreaTypeCode'] == 'CTY') & (capacity_df['MapCode'].isin(model_countries))]

In [58]:
capacity_by_country = capacity_df.groupby('MapCode') \
    .apply(lambda group: group.set_index('ProductionType')['AggregatedInstalledCapacity'].to_dict()).to_dict()

/tmp/ipykernel_17694/1427180708.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.set_index('ProductionType')['AggregatedInstalledCapacity'].to_dict()).to_dict()


In [59]:
all_capacity_types = {type for capacity_by_type in capacity_by_country.values() for type in capacity_by_type.keys()}
all_capacity_types

{'Biomass',
 'Fossil Brown coal/Lignite',
 'Fossil Coal-derived gas',
 'Fossil Gas',
 'Fossil Hard coal',
 'Fossil Oil',
 'Fossil Oil shale',
 'Fossil Peat',
 'Geothermal',
 'Hydro Pumped Storage',
 'Hydro Run-of-river and poundage',
 'Hydro Water Reservoir',
 'Marine',
 'Nuclear',
 'Other',
 'Other renewable',
 'Solar',
 'Waste',
 'Wind Offshore',
 'Wind Onshore'}

In [60]:
capacity_by_country['CH']

{'Hydro Run-of-river and poundage': 190.0,
 'Nuclear': 3373.0,
 'Hydro Water Reservoir': 4897.8,
 'Hydro Pumped Storage': 6228.0}

In [61]:
capacity_by_country['DE']

{'Fossil Brown coal/Lignite': 21262.0,
 'Biomass': 7080.25,
 'Fossil Hard coal': 27437.0,
 'Waste': 1685.0,
 'Geothermal': 40.22,
 'Other': 1421.0,
 'Hydro Water Reservoir': 1439.0,
 'Fossil Oil': 4614.0,
 'Wind Offshore': 4131.3,
 'Fossil Coal-derived gas': 0.0,
 'Fossil Gas': 32627.0,
 'Hydro Run-of-river and poundage': 4007.13,
 'Other renewable': 509.61,
 'Hydro Pumped Storage': 8894.24,
 'Nuclear': 10793.0,
 'Wind Onshore': 47042.0,
 'Solar': 40834.46}

### Generation

In [62]:
gen_df = pd.concat([pd.read_csv('%s/%d_%02d_AggregatedGenerationPerType_16.1.B_C.csv' % (entsoe_data_dir, year, month + 1),
                                sep='\t', usecols=['AreaTypeCode', 'MapCode', 'ProductionType', 'ActualGenerationOutput'])
                    for month in range(12)], ignore_index=True)
gen_df = gen_df[(gen_df['AreaTypeCode'] == 'CTY') & (gen_df['MapCode'].isin(model_countries))]
gen_df = gen_df.groupby(['MapCode', 'ProductionType']).filter(lambda g: g['ActualGenerationOutput'].count() > 1000)

In [63]:
gen_by_country = gen_df.groupby('MapCode') \
    .apply(lambda group: group.groupby('ProductionType')['ActualGenerationOutput'].mean().to_dict()).to_dict()

/tmp/ipykernel_17694/3656648501.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.groupby('ProductionType')['ActualGenerationOutput'].mean().to_dict()).to_dict()


In [64]:
all_gen_types = {type for gen_by_type in gen_by_country.values() for type in gen_by_type.keys()}
all_gen_types

{'Biomass',
 'Fossil Brown coal/Lignite',
 'Fossil Coal-derived gas',
 'Fossil Gas',
 'Fossil Hard coal',
 'Fossil Oil',
 'Fossil Oil shale',
 'Fossil Peat',
 'Geothermal',
 'Hydro Pumped Storage',
 'Hydro Run-of-river and poundage',
 'Hydro Water Reservoir',
 'Marine',
 'Nuclear',
 'Other',
 'Other renewable',
 'Solar',
 'Waste',
 'Wind Offshore',
 'Wind Onshore'}

Check that this is the same types as for the capacity computation:

In [65]:
assert all_gen_types == all_capacity_types

In [66]:
gen_by_country['CH']

{'Hydro Pumped Storage': 584.12845856102,
 'Hydro Run-of-river and poundage': 68.68743913857678,
 'Hydro Water Reservoir': 904.7273576958106,
 'Nuclear': 2244.7649134790527,
 'Solar': 38.301363013698634,
 'Wind Onshore': 7.819798534798534}

In [67]:
gen_by_country['DE']

{'Biomass': 4524.766928790984,
 'Fossil Brown coal/Lignite': 14842.226127903006,
 'Fossil Coal-derived gas': 431.8244763205829,
 'Fossil Gas': 2605.7666140141164,
 'Fossil Hard coal': 9208.768320241348,
 'Fossil Oil': 198.01513974271404,
 'Geothermal': 19.224431067850638,
 'Hydro Pumped Storage': 977.6436233492715,
 'Hydro Run-of-river and poundage': 1964.5508122723134,
 'Hydro Water Reservoir': 93.27279684653917,
 'Nuclear': 9134.58341302368,
 'Other': 309.27984090391624,
 'Other renewable': 140.52764571948998,
 'Solar': 3932.0249339708566,
 'Waste': 625.344625170765,
 'Wind Offshore': 1376.8343317395263,
 'Wind Onshore': 7431.724159836066}

**Remark**

A few computed productions are above capacity for a given type. The problem will be solved by using aggregated types.

In [68]:
for country, gen_by_type in gen_by_country.items():
    if country in capacity_by_country:
        for type, value in gen_by_type.items():
            if type in capacity_by_country[country]:
                if value > capacity_by_country[country][type]:
                    print('Production (%.2f) above capacity (%.2f) for type "%s" in country %s'
                          % (value, capacity_by_country[country][type], type, country))

Production (6.93) above capacity (0.00) for type "Fossil Oil" in country CZ
Production (109.33) above capacity (0.00) for type "Other" in country CZ
Production (431.82) above capacity (0.00) for type "Fossil Coal-derived gas" in country DE
Production (380.71) above capacity (0.00) for type "Fossil Brown coal/Lignite" in country ES
Production (284.76) above capacity (170.00) for type "Biomass" in country FR
Production (0.27) above capacity (0.00) for type "Fossil Oil" in country GR
Production (10472.35) above capacity (1708.00) for type "Other" in country IT
Production (7.21) above capacity (2.00) for type "Biomass" in country LU
Production (11.55) above capacity (1.00) for type "Solar" in country LU
Production (3912.27) above capacity (1.00) for type "Other" in country NL


### Aggregation

Aggregate production types:

In [69]:
entsoe_types_aggregation_map = {
    'Fossil Brown coal/Lignite'       : 'coal',
    'Fossil Hard coal'                : 'coal',
    'Fossil Coal-derived gas'         : 'gas',
    'Fossil Gas'                      : 'gas',
    'Fossil Oil'                      : 'gas',
    'Fossil Oil shale'                : 'gas',
    'Waste'                           : 'gas',
    'Fossil Peat'                     : 'coal',
    'Hydro Pumped Storage'            : 'hydro_storage',
    'Hydro Water Reservoir'           : 'hydro_storage',
    'Hydro Run-of-river and poundage' : 'hydro_ror',
    'Nuclear'                         : 'nuclear',
    'Biomass'                         : 'other',
    'Geothermal'                      : 'other',
    'Marine'                          : 'other',
    'Other'                           : 'other',
    'Other renewable'                 : 'other',
    'Solar'                           : 'other',
    'Wind Offshore'                   : 'other',
    'Wind Onshore'                    : 'other'
}

Check that all production types are mapped:

In [70]:
assert set(entsoe_types_aggregation_map.keys()) == all_gen_types

Check that the mapped types are the same as the aggregated types defined above, with the exception of "hydro":

In [71]:
assert set(entsoe_types_aggregation_map.values()) == aggregated_types - {'hydro'}

Compute aggregated capacity by country:

In [72]:
aggregated_capacity_by_country = {}

for country, capacity_by_type in capacity_by_country.items():
    types = {entsoe_types_aggregation_map[t] for t in capacity_by_type.keys()}
    
    aggregated_capacity_by_type = {t: 0.0 for t in types}
    for type, value in capacity_by_type.items():
        aggregated_capacity_by_type[entsoe_types_aggregation_map[type]] += value

    hydro = 0.0
    if 'hydro_storage' in types:
        hydro += aggregated_capacity_by_type['hydro_storage']
    if 'hydro_ror' in types:
        hydro += aggregated_capacity_by_type['hydro_ror']
    if hydro > 0.0:
        aggregated_capacity_by_type['hydro'] = hydro

    aggregated_capacity_by_country[country] = aggregated_capacity_by_type

In [73]:
aggregated_gen_by_country = {}

for country, gen_by_type in gen_by_country.items():
    types = {entsoe_types_aggregation_map[t] for t in gen_by_type.keys()}
    
    aggregated_gen_by_type = {t: 0.0 for t in types}
    for type, value in gen_by_type.items():
        aggregated_gen_by_type[entsoe_types_aggregation_map[type]] += value

    hydro = 0.0
    if 'hydro_storage' in types:
        hydro += aggregated_gen_by_type['hydro_storage']
    if 'hydro_ror' in types:
        hydro += aggregated_gen_by_type['hydro_ror']
    if hydro > 0.0:
        aggregated_gen_by_type['hydro'] = hydro

    aggregated_gen_by_country[country] = aggregated_gen_by_type

### Capacity usage

Compute usage:

In [74]:
usage_by_country = {country: {type: aggregated_gen_by_country[country][type] / value
                              for type, value in capacity_by_type.items() if type in aggregated_gen_by_country[country]}
                    for country, capacity_by_type in aggregated_capacity_by_country.items() if country in aggregated_gen_by_country}

In [75]:
for country, usage_by_type in usage_by_country.items():
    for value in usage_by_type.values():
        assert value <= 1.0

In [76]:
usage_by_country['CH']

{'hydro_storage': 0.13382011327336737,
 'hydro_ror': 0.36151283757145675,
 'nuclear': 0.665509906160407,
 'hydro': 0.13764322941333423}

In [77]:
usage_by_country['DE']

{'other': 0.17548570983031953,
 'hydro_storage': 0.10363800900741786,
 'hydro_ror': 0.49026380783062024,
 'coal': 0.4938703966846209,
 'gas': 0.09918694074007549,
 'nuclear': 0.8463433163183248,
 'hydro': 0.21167286705071936}

### Sum across countries

In [78]:
total_aggregated_capacity = {type: 0.0 for type in aggregated_types}

for country, capacity_by_type in aggregated_capacity_by_country.items():
    for type, value in capacity_by_type.items():
        total_aggregated_capacity[type] += value

total_aggregated_capacity

{'other': 224067.88,
 'hydro_storage': 103691.79,
 'hydro_ror': 43677.45,
 'coal': 141963.96,
 'hydro': 147369.24,
 'gas': 192906.67,
 'nuclear': 103160.18999999999}

In [79]:
total_aggregated_gen = {type: 0.0 for type in aggregated_types}

for country, gen_by_type in aggregated_gen_by_country.items():
    for type, value in gen_by_type.items():
        total_aggregated_gen[type] += value

total_aggregated_gen

{'other': 59508.86335331817,
 'hydro_storage': 13963.413655659808,
 'hydro_ror': 18266.386970728978,
 'coal': 62853.32941117908,
 'hydro': 32229.800626388787,
 'gas': 36266.15376996444,
 'nuclear': 76172.51808017504}

In [80]:
total_aggregated_usage = {type: total_aggregated_gen[type] / value for type, value in total_aggregated_capacity.items()}
total_aggregated_usage

{'other': 0.26558408707806835,
 'hydro_storage': 0.13466267344463634,
 'hydro_ror': 0.41821092968405843,
 'coal': 0.44274144938742965,
 'hydro': 0.21870100318349195,
 'gas': 0.18799844385870348,
 'nuclear': 0.7383906338305023}

In [81]:
total_aggregated_usage = {type: total_aggregated_gen[type] / value for type, value in total_aggregated_capacity.items()}
total_aggregated_usage

{'other': 0.26558408707806835,
 'hydro_storage': 0.13466267344463634,
 'hydro_ror': 0.41821092968405843,
 'coal': 0.44274144938742965,
 'hydro': 0.21870100318349195,
 'gas': 0.18799844385870348,
 'nuclear': 0.7383906338305023}

### Update the model

In [82]:
for gen in model['gen'].values():
    usage = usage_by_country[gen['country']][gen['aggregated_type']] \
        if gen['country'] in usage_by_country and gen['aggregated_type'] in usage_by_country[gen['country']] \
        else total_aggregated_usage[gen['aggregated_type']]
    gen['pexp'] = gen['pmax'] * usage

# Export improved model

Export:

In [83]:
model_json = json.dumps(model)
with open('../europe.json', 'w') as f:
    f.write(model_json)